In [1]:
import pandas as pd 
import os
import numpy as np 

In [2]:
main_path='/mnt/c/Users/rajes/Desktop/ketosis_project/files'


In [3]:
sensor_values= os.listdir('/mnt/c/Users/rajes/Desktop/ketosis_project/files')[:5]

In [6]:
list_df=[]
for path in sensor_values:
    temp=pd.read_csv(os.path.join(main_path, path))
    temp['date']=  path.split('.')[0]
    temp['date']= pd.to_datetime(temp['date'], format="%m_%d_%Y")
    list_df.append(temp)
sensor_df= pd.concat(list_df)

In [13]:
pd.read_csv(os.path.join(main_path, os.listdir(main_path)[6])).columns

Index(['Index', 'Cow', 'Grp.', 'Gyn.', 'Lact.', 'DIM', 'Cond.', 'Cond..1',
       'Cond..2', 'Avg.', 'Weight', 'Avg..1', 'Daily', 'Daily.1', 'Daily.2',
       'Breed.', 'After', 'Flow rate', 'Flow rate.1', 'Flow rate.2',
       'Unnamed: 20', 'pen_id', 'date'],
      dtype='object')

In [56]:
cohort1_dates = pd.to_datetime([
    '01-03-2025', '01-11-2025', '01-17-2025',
     '12-20-2024','12-27-2024' 
], format='%m-%d-%Y')

# Assign cohort
sensor_df['cohort'] = sensor_df['date'].apply(
    lambda x: 1 if x in cohort1_dates else 2
)
dict_dates_cohort_1= {value: i for i, value in enumerate(cohort1_dates)}

sensor_df['day'] = np.where(
    sensor_df['cohort'] == 1,
    sensor_df['date'].map(dict_dates_cohort_1),
    np.nan
)

sensor_df['date'] = np.where(sensor_df['date'] == pd.Timestamp('2025-02-20'),
                             pd.Timestamp('2025-02-21'),
                             sensor_df['date'])

cohort2_dates= pd.to_datetime([
'2025-01-24',
 '2025-01-31', '2025-02-07', '2025-02-14',
 '2025-02-21'
], format='%Y-%m-%d')

dict_dates_cohort_2= { value:i for i, value in enumerate(cohort2_dates)}

sensor_df['day']= np.where(

    sensor_df['cohort']==2,
    sensor_df['date'].map(dict_dates_cohort_2),
    np.nan
)

### extracting the serum work

In [57]:
serum_report=pd.read_csv(os.path.join(main_path, os.listdir(main_path)[-1]))

In [58]:
serum_report

,DATE,COHORT,PEN,TREATMENT,COWID,DAY,DIM,LACT,TBIL (mg/dL),NEFA (meEq/L),HBUT (mg/dL),GLUC (mg/dL),GGT (U/L),CHOL (mg/dL),AST (U/L),ALT (U/L),ALP (U/L),ALB (TL=<0.5) *(g/dL)
0,1/31/2025,2,2,CONTROL,77,7,195,1,0.0,0.02,1.3,16,0,40,16,5,6,0.6
1,1/31/2025,2,6,TREATMENT,892,7,102,5,0.1,0.11,5,41,22,217,54,27,23,2.9
2,2/7/2025,2,6,TREATMENT,892,14,102,5,0.1,0.17,6.5,52,29,316,102,39,29,3.8
3,2/14/2025,2,6,TREATMENT,892,21,102,5,0.0,0.04,2.9,25,8,73,18,8,9,1.0
4,2/21/2025,2,6,TREATMENT,892,28,102,5,0.0,0.13,1.9,18,4,55,12,5,5,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,1/24/2025,2,2,CONTROL,50197,0,146,7,0.0,0.03,6.5,45,78,185,56,31,45,3.0
674,1/31/2025,2,2,CONTROL,50197,7,146,7,0.0,0.02,5.5,53,44,141,67,24,31,2.7
675,2/7/2025,2,2,CONTROL,50197,14,146,7,0.0,0.17,5.8,53,85,196,77,26,43,3.2
676,2/14/2025,2,2,CONTROL,50197,21,146,7,0.0,0.02,4.9,34,17,44,17,7,8,0.7


In [59]:
serum_report['date']= pd.to_datetime(serum_report['DATE'])

In [60]:
sensor_df['sensor_values'] = sensor_df['sensor_values'].fillna(sensor_df['sensor values'])


In [62]:
sensor_df['date']= pd.to_datetime(sensor_df['date'])

In [63]:
merged_df = sensor_df.merge(
    serum_report,
    left_on=['date', 'cowid'],
    right_on=['date', 'COWID'],
    how='inner'
)


In [64]:
merged_df

,cowid,sensor_values,date,sensor values,cohort,day,DATE,COHORT,PEN,TREATMENT,...,TBIL (mg/dL),NEFA (meEq/L),HBUT (mg/dL),GLUC (mg/dL),GGT (U/L),CHOL (mg/dL),AST (U/L),ALT (U/L),ALP (U/L),ALB (TL=<0.5) *(g/dL)
0,4877,7.65,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,0.1,0.27,10.7,36,17,194,50,23,41,3.5
1,7623,2.35,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,0.0,0.14,5.6,26,16,98,25,6,9,1.1
2,9669,7.84,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,0.0,0.18,4.3,32,15,182,45,19,18,2.5
3,9795,6.58,2025-01-31,NaN,2,NaN,1/31/2025,2,2,CONTROL,...,0.1,0.12,5.2,56,17,248,108,20,44,3.9
4,9828,4.67,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,0.2,0.08,6.3,23,9,121,31,12,20,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,10056,5.02,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,0.1,0.13,4.1,36,10,89,21,8,11,1.3
113,10085,0.73,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,0.2,0.21,5.7,61,23,249,55,21,41,3.3
114,10086,3.70,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,0.0,0.28,15.4,51,21,313,61,23,26,3.7
115,50013,2.65,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,0.1,0.12,7.3,63,64,247,66,34,36,3.7


In [65]:
merged_df[['sensor_values','NEFA (meEq/L)']].corr()

,sensor_values,NEFA (meEq/L)
sensor_values,1.000000,0.091695
NEFA (meEq/L),0.091695,1.000000


In [73]:
os.listdir(main_path)

['1_31_2025.csv',
 '1_3_2025.csv',
 '2_14_2025.csv',
 '2_20_2025.csv',
 '2_7_2025.csv',
 'afi_smaxtec.csv',
 'first_trial.csv',
 'second_trial.csv',
 'serum_report.csv']

In [74]:
afi_smaxtec= pd.read_csv(os.path.join(main_path, 'afi_smaxtec.csv'))

In [76]:
afi_smaxtec.columns

Index(['Unnamed: 0', 'date', 'animal_id', 'Grp.', 'DIM', 'Lact.',
       'Conductivity', 'Weight', 'Daily', 'Breed.', 'activity',
       'Temp Without Drinkcycles', 'Normal temperature', 'Heat index',
       'rumination min/24h', 'water_intake in l', 'cohort', 'Pen', 'Status'],
      dtype='object')

In [77]:
os.listdir(main_path)

['1_31_2025.csv',
 '1_3_2025.csv',
 '2_14_2025.csv',
 '2_20_2025.csv',
 '2_7_2025.csv',
 'afi_smaxtec.csv',
 'blood',
 'DHI MILK(Sheet1).csv',
 'DHI MILK(Sheet2).csv',
 'first_trial.csv',
 'second_trial.csv',
 'serum_report.csv']

In [78]:
milk_df= pd.read_csv(os.path.join(main_path, 'DHI MILK(Sheet1).csv'))

In [79]:
milk_df.columns

Index(['DATE', 'COHORT', 'PEN', 'TREATMENT', 'COWID', 'DAY', 'Fat', 'Protein',
       'Lactose', 'SNF', 'MUN', 'SCC', 'BHB', 'Acetone', 'Unnamed: 14',
       'Unnamed: 15', 'sick ', 'Check', 'no data '],
      dtype='object')

In [80]:
milk_df['date']= pd.to_datetime(milk_df['DATE'])

In [88]:
milk_df['cowid'] = milk_df['COWID'].astype(str).str.split(' ').str[0].astype('int64')


In [90]:
merged_milk_df= merged_df.merge(
    milk_df,
    left_on=['date', 'cowid'],
    right_on=['date', 'cowid'],
    how='inner'

)

In [91]:
merged_milk_df

,cowid,sensor_values,date,sensor values,cohort,day,DATE_x,COHORT_x,PEN_x,TREATMENT_x,...,SNF,MUN,SCC,BHB,Acetone,Unnamed: 14,Unnamed: 15,sick,Check,no data
0,4877,7.65,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,8.50,6.40361,43,0.96,0.00,NaN,NaN,NaN,NaN,NaN
1,7623,2.35,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,8.87,6.33637,210,1.23,0.03,NaN,NaN,NaN,NaN,NaN
2,9669,7.84,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,8.08,4.34631,4053,1.31,0.02,NaN,NaN,NaN,NaN,NaN
3,9795,6.58,2025-01-31,NaN,2,NaN,1/31/2025,2,2,CONTROL,...,8.10,2.49333,6463,0.51,0.00,NaN,NaN,NaN,NaN,NaN
4,9828,4.67,2025-01-31,NaN,2,NaN,1/31/2025,2,6,TREATMENT,...,9.01,9.03482,46,0.63,0.05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,10056,5.02,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,9.31,3.37343,37,0.94,0.06,NaN,NaN,NaN,NaN,NaN
111,10085,0.73,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,9.92,14.12919,1101,1.42,0.17,NaN,NaN,NaN,NaN,NaN
112,10086,3.70,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,8.60,6.79028,245,1.22,0.04,NaN,NaN,NaN,NaN,NaN
113,50013,2.65,2025-02-07,NaN,2,NaN,2/7/2025,2,5,TREATMENT,...,8.72,6.93888,160,1.37,0.06,NaN,NaN,NaN,NaN,NaN


In [93]:
merged_milk_df[['BHB','Acetone', 'sensor_values']].corr()

,BHB,Acetone,sensor_values
BHB,1.000000,0.218766,0.378146
Acetone,0.218766,1.000000,0.203165
sensor_values,0.378146,0.203165,1.000000
